In [144]:
# Configuration

import logging
import sys
import os
import pandas as pd
from datetime import datetime
from pathlib import Path


rightnow = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
LINEAGE_LEVEL = "family"
HMM_FILE = "ascogs_escrt_system_20260116_121403.tsv"
FLANK_K = 0               # number of 'other' tokens allowed at each flank

NOTEBOOK_PATH = Path.cwd() / "ESCRT_system_synteny_2.ipynb"

WINDOW = 5
CORE_TARGETS = ("vps", "escrt", "katanin", "eap", "flad")


MAIN_OUTDIR = os.path.join(os.getcwd(), f"ESCRT_synteny_pipeline_m3v2_{LINEAGE_LEVEL}_{rightnow}")
os.makedirs(MAIN_OUTDIR, exist_ok=True)

def setup_logging(log_file=None):
    """
    Configure logging for the pipeline.
    Logs to stdout and optionally to a file.
    """
    handlers = [logging.StreamHandler(sys.stdout)]
    if log_file:
        handlers.append(logging.FileHandler(log_file))

    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] %(message)s",
        handlers=handlers,
        force=True  # Force reconfiguration if already set
    )


STEP = 0

setup_logging(os.path.join(MAIN_OUTDIR, f"ESCRT_synteny_pipeline{rightnow}.log"))


In [145]:
import json

# ------------------------------------------------------------------
# Load the current notebook file (Jupyter notebooks are JSON)
# ------------------------------------------------------------------
with open(NOTEBOOK_PATH, "r", encoding="utf-8") as nb:
    notebook_json = json.load(nb)


# ------------------------------------------------------------------
# Remove execution outputs from code cells
#   - prevents massive logs
#   - avoids embedding binary blobs (plots, images)
#   - keeps only the executable logic + structure
# ------------------------------------------------------------------
def strip_outputs(nb):
    for cell in nb.get("cells", []):
        if cell.get("cell_type") == "code":
            cell["outputs"] = []
            cell["execution_count"] = None
    return nb

notebook_json = strip_outputs(notebook_json)


# ------------------------------------------------------------------
# Serialize notebook JSON to a formatted string
#   - indentation preserves readability in logs
#   - done once to avoid repeated JSON encoding
# ------------------------------------------------------------------
notebook_str = json.dumps(notebook_json, indent=2)


# ------------------------------------------------------------------
# Append notebook snapshot to the log file
#   - logged line-by-line to preserve formatting
#   - ensures compatibility with logging handlers
# ------------------------------------------------------------------
logging.info("=" * 80)
logging.info("NOTEBOOK JSON SNAPSHOT (Start of Run)")
logging.info("=" * 80)

for line in notebook_str.splitlines():
    logging.info(line)


2026-01-16 20:11:34,290 [INFO] ================================================================================
2026-01-16 20:11:34,291 [INFO] NOTEBOOK JSON SNAPSHOT (Start of Run)
2026-01-16 20:11:34,291 [INFO] ================================================================================
2026-01-16 20:11:34,292 [INFO] {
2026-01-16 20:11:34,293 [INFO]   "cells": [
2026-01-16 20:11:34,293 [INFO]     {
2026-01-16 20:11:34,294 [INFO]       "cell_type": "code",
2026-01-16 20:11:34,294 [INFO]       "execution_count": null,
2026-01-16 20:11:34,295 [INFO]       "id": "99fd6d19",
2026-01-16 20:11:34,295 [INFO]       "metadata": {},
2026-01-16 20:11:34,295 [INFO]       "outputs": [],
2026-01-16 20:11:34,296 [INFO]       "source": [
2026-01-16 20:11:34,296 [INFO]         "# Configuration\n",
2026-01-16 20:11:34,297 [INFO]         "\n",
2026-01-16 20:11:34,297 [INFO]         "import logging\n",
2026-01-16 20:11:34,297 [INFO]         "import sys\n",
2026-01-16 20:11:34,298 [INFO]         "impor

In [146]:
# Merge HMM hits with protein metadata


# ----------------------------
# Input files
# ----------------------------
hits_file = "/home/anirudh/synteny/hmms/ESCRT_results/hits/ESCRT_hits_final.csv"
proteins_file = "/home/anirudh/synteny/proteins_genomes_cp90_con5.csv"

# ----------------------------
# Output file naming
# ----------------------------
hits_file_name = os.path.basename(hits_file).split(".")[0]
STEP += 1
merge_file_name = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]{hits_file_name}_merged_{rightnow}.csv")

logging.info("HMM hits file: %s", hits_file)
logging.info("Protein metadata file: %s", proteins_file)
logging.info("Merged output file: %s", merge_file_name)

# ----------------------------
# Load HMM hits
# ----------------------------
try:
    hits = pd.read_csv(hits_file)
except Exception as e:
    logging.error("Failed to load HMM hits file: %s", e)
    raise

hits.columns = hits.columns.str.strip()
logging.info("HMM hits columns: %s", list(hits.columns))
logging.info("HMM hits rows: %d", len(hits))

# ----------------------------
# Load protein metadata
# ----------------------------
try:
    proteins = pd.read_csv(proteins_file)
except Exception as e:
    logging.error("Failed to load protein metadata file: %s", e)
    raise

proteins.columns = proteins.columns.str.strip()
logging.info("Protein metadata columns: %s", list(proteins.columns))
logging.info("Protein metadata rows: %d", len(proteins))

# ----------------------------
# Sanity checks before merge
# ----------------------------
required_hits_cols = {"target"}
required_prot_cols = {"cds_id"}

missing_hits = required_hits_cols - set(hits.columns)
missing_prots = required_prot_cols - set(proteins.columns)

if missing_hits:
    logging.error("Missing required columns in HMM hits file: %s", missing_hits)
    raise ValueError(f"Missing columns in hits file: {missing_hits}")

if missing_prots:
    logging.error("Missing required columns in protein metadata file: %s", missing_prots)
    raise ValueError(f"Missing columns in proteins file: {missing_prots}")

logging.info("Input sanity checks passed")

# ----------------------------
# Merge HMM hits with protein metadata
# ----------------------------
logging.info("Merging HMM hits with protein metadata")
logging.info("Rows before merge: hits=%d, proteins=%d", len(hits), len(proteins))

merged = pd.merge(
    hits,
    proteins,
    left_on="target",
    right_on="cds_id",
    how="inner"
)

logging.info("Rows after merge: %d", len(merged))
logging.info("Unique targets merged: %d", merged["target"].nunique())

# ----------------------------
# Write output
# ----------------------------
try:
    
    merged.to_csv(merge_file_name, index=False)
except Exception as e:
    logging.error("Failed to write merged output file %s: %s", merge_file_name, e)
    raise

logging.info("Merged output written to: %s", merge_file_name)


2026-01-16 20:11:34,799 [INFO] HMM hits file: /home/anirudh/synteny/hmms/ESCRT_results/hits/ESCRT_hits_final.csv
2026-01-16 20:11:34,800 [INFO] Protein metadata file: /home/anirudh/synteny/proteins_genomes_cp90_con5.csv
2026-01-16 20:11:34,801 [INFO] Merged output file: /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_m3v2_family_2026-01-16-20-11-34/[STEP:1]ESCRT_hits_final_merged_2026-01-16-20-11-34.csv
2026-01-16 20:11:34,828 [INFO] HMM hits columns: ['target', 'tacc', 'tlen', 'query', 'qacc', 'qlen', 'full_evalue', 'full_score', 'full_bias', 'dom_idx', 'dom_count', 'c_evalue', 'i_evalue', 'dom_score', 'dom_bias', 'hmm_from', 'hmm_to', 'ali_from', 'ali_to', 'env_from', 'env_to', 'acc', 'description', 'source_file']
2026-01-16 20:11:34,828 [INFO] HMM hits rows: 24597
2026-01-16 20:11:35,916 [INFO] Protein metadata columns: ['Name', 'Completeness', 'Contamination', 'Completeness_Model_Used', 'Translation_Table_Used', 'Coding_Density', 'Contig_N50', 'Average_Gene_Length', 'Genome_Size'

Merge with the ascogs definition

In [147]:
# Merge HMM hits with AsCOG annotations

# ----------------------------
# Load AsCOG annotation table
# ----------------------------
ascogs_tsv = f"/home/anirudh/synteny/hmms/Sources/{HMM_FILE}"
logging.info("Loading AsCOGs from: %s", ascogs_tsv)

try:
    ascogs = pd.read_csv(ascogs_tsv, sep="\t")
except Exception as e:
    logging.error("Failed to load AsCOGs file: %s", e)
    raise

ascogs.columns = ascogs.columns.str.strip()

logging.info("AsCOGs columns: %s", list(ascogs.columns))
logging.info("AsCOGs rows: %d", len(ascogs))

# ----------------------------
# Sanity checks
# ----------------------------
required_cols = {"ascog_id"}
missing = required_cols - set(ascogs.columns)

if missing:
    logging.error("Missing required columns in AsCOGs file: %s", missing)
    raise ValueError(f"Missing required columns: {missing}")

logging.info("AsCOGs sanity check passed")

# ----------------------------
# Merge HMM hits with AsCOGs
# ----------------------------
logging.info("Merging HMM hits with AsCOG annotations")
logging.info("HMM hits rows before merge: %d", len(merged))

merged_ascogs = pd.merge(
    merged,
    ascogs,
    left_on="query",
    right_on="ascog_id",
    how="left"
)

logging.info("Rows after AsCOG merge: %d", len(merged_ascogs))
logging.info("Unique AsCOGs matched: %d", merged_ascogs["ascog_id"].nunique())
logging.info("Unique proteins matched: %d", merged_ascogs["target"].nunique())

# ----------------------------
# Write output
# ----------------------------
STEP += 1
final_output_file = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]ESCRT_hits_final_merged_{rightnow}_ascog.csv")

merged_ascogs = merged_ascogs[["target", "tacc", "tlen", "query", "qacc", "qlen", "c_evalue", "i_evalue", "dom_score",  "hmm_from", "hmm_to", "ali_from", "ali_to", "env_from", "env_to", "acc", "Name", "Completeness", "Contamination", "Contig_N50", "Total_Contigs", "organism_name", "cds_id", "header", "gene", "description_y"]]

try:
    merged_ascogs.to_csv(final_output_file, index=False)
except Exception as e:
    logging.error("Failed to write output file %s: %s", final_output_file, e)
    raise

logging.info("Final merged file written to: %s", final_output_file)


2026-01-16 20:11:36,315 [INFO] Loading AsCOGs from: /home/anirudh/synteny/hmms/Sources/ascogs_escrt_system_20260116_121403.tsv
2026-01-16 20:11:36,317 [INFO] AsCOGs columns: ['ascog_id', 'arcog_id', 'category', 'gene', 'description']
2026-01-16 20:11:36,318 [INFO] AsCOGs rows: 89
2026-01-16 20:11:36,318 [INFO] AsCOGs sanity check passed
2026-01-16 20:11:36,319 [INFO] Merging HMM hits with AsCOG annotations
2026-01-16 20:11:36,320 [INFO] HMM hits rows before merge: 24597
2026-01-16 20:11:36,329 [INFO] Rows after AsCOG merge: 24597
2026-01-16 20:11:36,331 [INFO] Unique AsCOGs matched: 88
2026-01-16 20:11:36,333 [INFO] Unique proteins matched: 7285
2026-01-16 20:11:36,468 [INFO] Final merged file written to: /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_m3v2_family_2026-01-16-20-11-34/[STEP:2]ESCRT_hits_final_merged_2026-01-16-20-11-34_ascog.csv


In [148]:
# Select best hits based on e-value and coverage

merged_ascogs["coverage"] = (
    (merged_ascogs["hmm_to"] - merged_ascogs["hmm_from"] + 1)
    / merged_ascogs["qlen"]
)

merged_ascogs = (
    merged_ascogs
    .sort_values(
        by=["target", "query", "i_evalue", "coverage", "dom_score"],
        ascending=[True, True, True, False, False]
    )
)
best_hits = (
    merged_ascogs
    .drop_duplicates(subset=["target", "query"], keep="first")
)

best_hits = best_hits[(best_hits["i_evalue"] <= 1e-5) & (best_hits["coverage"] >= 0.65)]

best_hits = best_hits[["target","query","gene","dom_score","i_evalue","coverage","description_y","tacc","tlen","qacc","qlen","c_evalue","hmm_from","hmm_to","ali_from","ali_to","env_from","env_to","acc","Name","Completeness","Contamination","Contig_N50","Total_Contigs","organism_name"]]

STEP += 1
best_hits_file = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]ESCRT_hits_final_merged_{rightnow}_ascog_best_hits.csv")

best_hits.to_csv(best_hits_file, index=False)
logging.info("Best hits file written to: %s", best_hits_file)


2026-01-16 20:11:36,533 [INFO] Best hits file written to: /home/anirudh/synteny/hmms/ESCRT_synteny_pipeline_m3v2_family_2026-01-16-20-11-34/[STEP:3]ESCRT_hits_final_merged_2026-01-16-20-11-34_ascog_best_hits.csv


In [149]:
# Compute protein counts with AsCOG hits


protein_counts = (
    best_hits
    .dropna(subset=["gene", "query"])
    .assign(
        gene=lambda df: df["gene"].str.strip(),
        ascog_id=lambda df: df["query"].str.strip()
    )
    .groupby("target")
    .agg(
        ascog_hits=("gene", lambda x: list(x.unique())),
        ascog_ids=("query", lambda x: list(x.unique()))
    )
    .reset_index()
    .assign(
        ascog_count=lambda df: df["ascog_hits"].apply(len)
    )
    .sort_values(by = ["ascog_count"], ascending=[False])
    .assign(
        arch_str=lambda d: d["ascog_hits"].apply(lambda x: "+".join(sorted(x)))
    )
    .sort_values(
        by=["ascog_count", "arch_str"],
        ascending=[False, True]
    )
)

logging.info("Computed protein counts with AsCOG hits")

STEP += 1
protein_counts_file = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]protein_ascog_counts_{rightnow}.csv")
try:
    protein_counts.to_csv(protein_counts_file, index=False)
except Exception as e:
    logging.error("Failed to write protein counts file %s: %s", protein_counts_file, e)
    raise ValueError(f"Failed to write protein counts file: {e}")

2026-01-16 20:11:36,621 [INFO] Computed protein counts with AsCOG hits


In [150]:
# Infer architectures based on rules

# Order matters: first match wins
ARCH_RULES = [
    # Vps23 canonical
    ({"CC-Vps23", "PH-Vps23", "Vps23"}, "CC-PH-Vps23"),
    ({"CC-Vps23", "PH-Vps23"}, "CC-PH-Vps23"),

    # E2–Vps23 fusion
    ({"E2-Vps23", "E2", "Vps23"}, "E2-Vps23"),
    ({"E2-Vps23", "E2"}, "E2-Vps23"),
    ({"E2-Vps23", "Vps23"}, "E2-Vps23"),

    # ESCRT-I–linked E2
    ({"Vps28", "E2-Vps23", "E2"}, "E2-ESCRT-I"),

    # ESCRT-II Vps22-Vps36 fusion
    ({"CC-Vps23", "EAP30"}, "Vps23-EAP30"),

    # MPN-E3_doms
    ({"MPN", "E3-dom"}, "MPN-E3-dom"),
]



import pandas as pd

def infer_architecture(group: pd.DataFrame) -> pd.Series:
    """
    Infer a single architecture for one protein (target)
    using explicit rules, else fallback to best i-Evalue hit.
    """

    # Clean gene names
    genes = set(
        group["gene"]
        .dropna()
        .astype(str)
        .str.strip()
    )

    # 1. Apply rule-based architecture inference
    for rule_genes, architecture in ARCH_RULES:
        if rule_genes.issubset(genes):
            return pd.Series({
                "architecture": architecture,
                "architecture_method": "rule",
                "architecture_components": ",".join(sorted(rule_genes))
            })

    # 2. Fallback: best single hit by independent E-value
    best_hit = (
        group
        .dropna(subset=["i_evalue"])
        .sort_values("i_evalue", ascending=True)
        .iloc[0]
    )

    return pd.Series({
        "architecture": best_hit["gene"],
        "architecture_method": "best_i_evalue",
        "architecture_components": best_hit["gene"]
    })



architecture_df = (
    best_hits
    .dropna(subset=["target", "gene", "i_evalue"])
    .groupby("target", group_keys=False)
    .apply(infer_architecture)
    .reset_index()
)



df_with_arch = best_hits.merge(
    architecture_df,
    on="target",
    how="left"
)


df_with_arch = df_with_arch[["target","Name","Completeness","Contamination","Contig_N50","Total_Contigs","organism_name","architecture","architecture_method","architecture_components"]]

df_with_arch.drop_duplicates(inplace=True)
logging.info("Inferred architectures for proteins")

STEP += 1
df_with_arch_file = os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}]ESCRT_hits_with_architectures_{rightnow}.csv")
try:
    df_with_arch.to_csv(df_with_arch_file, index=False)
except Exception as e: 
    logging.error("Failed to write architecture output file %s: %s", df_with_arch_file, e)
    raise ValueError(f"Failed to write architecture output file: {e}")

2026-01-16 20:11:38,680 [INFO] Inferred architectures for proteins


/tmp/ipykernel_212142/1008619236.py:71: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(infer_architecture)


In [151]:

# ------------------------------------------------------------
# Parse Prodigal GFF and assign gene order PER CONTIG
# ------------------------------------------------------------
def parse_prodigal_gff(gff_path, genome_id):
    """
    Parse a Prodigal GFF3 file and extract CDS gene order per contig.

    Gene order is defined by appearance order in the GFF,
    which corresponds to genomic order for Prodigal output.
    
    Args:
        gff_path: Path to the GFF file
        genome_id: Identifier for the genome
        
    Returns:
        DataFrame with columns: genome_id, contig, gene_index, 
                                protein_id, start, end, strand
    """

    rows = []
    gene_counter = {}  # separate gene index per contig

    logging.info(f"Parsing GFF: {gff_path.name}")

    with open(gff_path) as f:
        for line in f:
            # Skip comment lines
            if line.startswith("#"):
                continue

            parts = line.rstrip().split("\t")
            if len(parts) != 9:
                continue

            contig, source, feature, start, end, score, strand, phase, attrs = parts

            # Only process CDS features
            if feature != "CDS":
                continue

            # Parse attributes to extract protein ID
            attr_dict = {}
            for item in attrs.split(";"):
                if "=" in item:
                    k, v = item.split("=", 1)
                    attr_dict[k] = v

            protein_id = attr_dict.get("ID")
            if protein_id is None:
                continue

            # Increment gene index PER CONTIG (not genome-wide)
            gene_counter.setdefault(contig, 0)
            gene_counter[contig] += 1

            rows.append({
                "genome_id": genome_id,
                "contig": contig,
                "gene_index": gene_counter[contig],
                "protein_id": protein_id,
                "start": int(start),
                "end": int(end),
                "strand": strand
            })

    logging.info(f"  → Parsed {len(rows)} CDS features across {len(gene_counter)} contigs")
    return pd.DataFrame(rows)


    

# ------------------------------------------------------------
# Load only the GFFs required by the hits table
# ------------------------------------------------------------
def load_gffs_from_hits(hits_df, gff_dir):
    """
    Parse GFF files only for genomes present in the hits dataframe.
    GFF filenames are inferred as: <genome_file>_genomic/<genome_file>_genomic.gff
    
    Args:
        hits_df: DataFrame containing hits with 'genome_file' column
        gff_dir: Directory containing GFF files
        
    Returns:
        Combined DataFrame of all parsed GFF files
    """

    all_gff_rows = []
    
    unique_genomes = hits_df["Name"].unique()
    logging.info(f"Loading GFFs for {len(unique_genomes)} unique genomes")

    for genome_id in unique_genomes:
        gff_path = gff_dir / f"{genome_id}_genomic" / f"{genome_id}_genomic.gff"

        if not gff_path.exists():
            logging.warning(f"Missing GFF: {gff_path}")
            continue

        gff_df = parse_prodigal_gff(gff_path, genome_id)
        all_gff_rows.append(gff_df)

    if not all_gff_rows:
        logging.error("No GFF files were successfully loaded.")
        sys.exit(1)

    combined_gff = pd.concat(all_gff_rows, ignore_index=True)
    logging.info(f"Total CDS features loaded: {len(combined_gff)}")
    
    return combined_gff

def compress_others(tokens, threshold=10):
    """
    Compress long runs of 'other' into 'other[n]' if n > threshold.
    """
    compressed = []
    i = 0

    while i < len(tokens):
        if tokens[i] != "other":
            compressed.append(tokens[i])
            i += 1
            continue

        # count run
        j = i
        while j < len(tokens) and tokens[j] == "other":
            j += 1

        run_len = j - i

        if run_len > threshold:
            compressed.append(f"other[{run_len}]")
        else:
            compressed.extend(["other"] * run_len)

        i = j

    return compressed

def extract_neighborhoods(anchor_df, gff_df, window=5):
    """
    Extract genomic neighborhoods around ESCRT-related anchor genes.
    """
    keywords = CORE_TARGETS

    logging.info(f"Starting neighborhood extraction (±{window} genes)")
    logging.info(f"Initial anchors: {len(anchor_df)}")

    # Filter anchors by keywords
    filtered_anchors = anchor_df[
        anchor_df["architecture"]
        .astype(str)
        .str.lower()
        .str.contains("|".join(keywords))
    ].copy()

    logging.info(f"Anchors after ESCRT filter: {len(filtered_anchors)}")

    if filtered_anchors.empty:
        logging.error("No ESCRT-related anchors found")
        sys.exit(1)

    neighborhoods = []

    # Group by genome and contig
    for (genome, contig), anchors in filtered_anchors.groupby(
        ["genome_id", "contig"]
    ):
        anchor_indices = anchors["gene_index"].values
        
        # Define window boundaries
        start = anchor_indices.min() - window
        end = anchor_indices.max() + window

        # Extract all genes in the window from gff_df
        block = gff_df[
            (gff_df["genome_id"] == genome) &
            (gff_df["contig"] == contig) &
            (gff_df["gene_index"].between(start, end))
        ].copy()

        if block.empty:
            continue

        # Sort by gene position
        block = block.sort_values("gene_index").reset_index(drop=True)

        # Create a mapping of protein_id to architecture from ALL anchors
        # (not just filtered ones), so we get complete architecture info
        arch_map = anchor_df.set_index("protein_id")["architecture"].to_dict()
        
        # Apply architecture: use anchor architecture if available, else "other"
        block["architecture"] = block["protein_id"].map(arch_map).fillna("other")

        # Build architecture token list for the ENTIRE neighborhood
        tokens = block["architecture"].tolist()
        compressed_tokens = compress_others(tokens, threshold=10)


        # Add metadata about the neighborhood
        block["num_anchors"] = len(anchors)

        block["window_start"] = start
        block["window_end"] = end
        block["neighborhood_architecture_compressed"] = ",".join(compressed_tokens)
        block["position_in_neighborhood"] = range(len(block))

        neighborhoods.append(block)

    if not neighborhoods:
        logging.error("No neighborhoods extracted after processing")
        sys.exit(1)

    combined = pd.concat(neighborhoods, ignore_index=True)

    logging.info(
        f"Extracted {len(combined)} genes from "
        f"{len(neighborhoods)} contig neighborhoods"
    )

    return combined

# ------------------------------------------------------------
# Load GTDB taxonomy
# ------------------------------------------------------------
def load_gtdb_taxonomy(gtdb_tsv):
    """
    Load GTDB taxonomy file and split into rank-specific columns.
    
    GTDB format: genome_id\td__Domain;p__Phylum;c__Class;o__Order;f__Family;g__Genus;s__Species
    
    Args:
        gtdb_tsv: Path to GTDB taxonomy TSV file
        
    Returns:
        DataFrame with columns: genome_id, domain, phylum, class, order, family, genus, species
    """
    logging.info(f"Loading GTDB taxonomy from: {gtdb_tsv}")
    
    # Read the taxonomy file
    tax_df = pd.read_csv(
        gtdb_tsv,
        sep="\t",
        header=None,
        names=["genome_id", "taxonomy"],
        dtype=str
    )
    
    tax_df["genome_id_base"] = tax_df["genome_id"].str.split("_", n=1).str[1]
    logging.info(f"Loaded taxonomy for {len(tax_df)} genomes")
    logging.info(f"Sample taxonomy entry: {tax_df['taxonomy'].iloc[0]}")
    
    # Split taxonomy string by semicolons
    tax_split = tax_df["taxonomy"].str.split(";", expand=True)
    logging.info(f"Taxonomy split into {tax_split.shape[1]} columns")
    
    # Map column indices to taxonomic ranks
    rank_map = {
        0: "domain",
        1: "phylum",
        2: "class",
        3: "order",
        4: "family",
        5: "genus",
        6: "species"
    }
    
    # Extract each rank and remove the prefix (e.g., "d__", "p__")
    for idx, rank in rank_map.items():
        if idx < tax_split.shape[1]:
            # Remove the rank prefix using regex (e.g., "p__" from "p__Crenarchaeota")
            tax_df[rank] = tax_split[idx].str.replace(r"^[a-z]__", "", regex=True)
            logging.info(f"Extracted {rank}: {tax_df[rank].notna().sum()} non-null values")
        else:
            tax_df[rank] = pd.NA
            logging.warning(f"Column {idx} for rank '{rank}' not found in taxonomy data")
    
    # Drop the original concatenated taxonomy string
    tax_df.drop(columns=["taxonomy"], inplace=True)
    
    # Log sample of parsed taxonomy
    logging.info(f"Sample parsed taxonomy:")
    logging.info(tax_df[["genome_id", "domain", "phylum", "class"]].head().to_string())
    
    return tax_df


# ------------------------------------------------------------
# Merge taxonomy into neighborhood data
# ------------------------------------------------------------
def merge_taxonomy(escrt_csv, gtdb_tsv, out_csv):
    """
    Merge GTDB taxonomy into ESCRT neighborhood dataframe.
    
    Args:
        escrt_csv: Path to ESCRT neighborhoods CSV
        gtdb_tsv: Path to GTDB taxonomy TSV
        out_csv: Output path for merged CSV
        
    Returns:
        Merged DataFrame with taxonomy columns added
    """
    logging.info("=" * 70)
    logging.info("STARTING TAXONOMY MERGE")
    logging.info("=" * 70)
    
    # Load neighborhood data
    logging.info(f"Loading neighborhood data from: {escrt_csv}")
    escrt_df = pd.read_csv(escrt_csv)
    escrt_df['genome_id_base'] = (
    escrt_df['genome_id']
    .str.split("_", n=2)
    .str[0:2]
    .str.join("_")
)

    logging.info(f"Neighborhood data: {len(escrt_df)} rows, {len(escrt_df['genome_id'].unique())} unique genomes")
    
    # Load taxonomy data
    tax_df = load_gtdb_taxonomy(gtdb_tsv)
    
    # Check for overlap between datasets
    escrt_genomes = set(escrt_df["genome_id_base"].unique())
    tax_genomes = set(tax_df["genome_id_base"].unique())
    overlap = escrt_genomes.intersection(tax_genomes)
    
    logging.info(f"Genome ID overlap check:")
    logging.info(f"  Genomes in neighborhood data: {len(escrt_genomes)}")
    logging.info(f"  Genomes in taxonomy data: {len(tax_genomes)}")
    logging.info(f"  Overlapping genomes: {len(overlap)}")
    
    if len(overlap) == 0:
        logging.error("NO OVERLAP between genome IDs!")
        logging.error(f"Sample neighborhood genome IDs: {list(escrt_genomes)[:5]}")
        logging.error(f"Sample taxonomy genome IDs: {list(tax_genomes)[:5]}")
        logging.error("Check if genome_id formats match between files!")
        sys.exit(1)
    
    if len(overlap) < len(escrt_genomes):
        missing = len(escrt_genomes) - len(overlap)
        logging.warning(f"{missing} genomes from neighborhood data not found in taxonomy!")
    
    # Perform the merge
    logging.info("Performing left join on genome_id...")
    merged = escrt_df.merge(
        tax_df,
        on="genome_id_base",
        how="left"
    )
    
    # Validate merge results
    logging.info(f"Merge complete: {len(merged)} rows")
    
    # Check how many rows got taxonomy data
    tax_columns = ["domain", "phylum", "class", "order", "family", "genus", "species"]
    for col in tax_columns:
        non_null = merged[col].notna().sum()
        pct = (non_null / len(merged)) * 100
        logging.info(f"  {col}: {non_null} non-null ({pct:.1f}%)")
    
    # Save to CSV
    logging.info(f"Saving merged data to: {out_csv}")
    merged.to_csv(out_csv, index=False)
    
    # Show sample of merged data
    logging.info("Sample of merged data with taxonomy:")
    sample_cols = ["genome_id", "protein_id", "domain", "phylum", "class", "order"]
    available_cols = [col for col in sample_cols if col in merged.columns]
    logging.info(merged[available_cols].head(10).to_string())
    
    logging.info("=" * 70)
    logging.info("TAXONOMY MERGE COMPLETE")
    logging.info("=" * 70)
    
    return merged




In [152]:
from pathlib import Path
import os
import sys
import logging

# ------------------------------------------------------------
# SYNTENY PIPELINE (NOTEBOOK / LINEAR EXECUTION)
# ------------------------------------------------------------

logging.info("=" * 70)
logging.info("STARTING SYNTENY PIPELINE")
logging.info("=" * 70)

logging.info(f"Hits DataFrame: {df_with_arch.shape}")
logging.info(f"GFF directory: /home/anirudh/genomes/selected_genomes/prokka_results")
logging.info(f"Window size: ±{WINDOW} genes")

gff_dir = Path("/home/anirudh/genomes/selected_genomes/prokka_results")

# --------------------------------------------------------
# STEP 1: Load GFFs
# --------------------------------------------------------
logging.info("\n[STEP 1] Loading GFF files...")

gff_df = load_gffs_from_hits(df_with_arch, gff_dir)


gff_df_file = os.path.join(
    MAIN_OUTDIR,
    f"[STEP:{STEP}.5]gff_dataframe_{rightnow}.csv"
)
gff_df.to_csv(gff_df_file, index=False)
logging.info("Saved GFF dataframe: %s", gff_df_file)

# --------------------------------------------------------
# STEP 2: Map anchor hits to gene order
# --------------------------------------------------------
logging.info("\n[STEP 2] Mapping hits to gene order...")

anchor_df = df_with_arch.merge(
    gff_df,
    left_on="target",
    right_on="protein_id",
    how="inner"
)

if anchor_df.empty:
    logging.error("No hits could be mapped to GFFs (protein ID mismatch?)")
    sys.exit(1)

logging.info("Mapped %d anchor hits to GFF entries", len(anchor_df))

logging.info("Saving anchor hits...")
STEP += 1

anchor_hits_file = os.path.join(
    MAIN_OUTDIR,
    f"[STEP:{STEP}]escrt_anchor_hits_{rightnow}.csv"
)
anchor_df.to_csv(anchor_hits_file, index=False)
logging.info("Saved: %s", anchor_hits_file)

# --------------------------------------------------------
# STEP 3: Extract contig architectures / neighborhoods
# --------------------------------------------------------
logging.info("\n[STEP 3] Extracting contig architectures...")

neigh_df = extract_neighborhoods(
    anchor_df=anchor_df,
    gff_df=gff_df,
    window=WINDOW
)

logging.info("Extracted %d neighborhood gene rows", len(neigh_df))

logging.info("=" * 70)
logging.info("SYNTENY PIPELINE COMPLETED SUCCESSFULLY")
logging.info("=" * 70)

# --------------------------------------------------------
# SANITY CHECKS
# --------------------------------------------------------
logging.info("\n" + "=" * 70)
logging.info("PIPELINE OUTPUT SUMMARY")
logging.info("=" * 70)

logging.info("\nAnchor gene architecture distribution:")
logging.info(anchor_df["architecture"].value_counts().to_string())

logging.info("\nFirst few neighborhood entries:")
logging.info(neigh_df.head().to_string())

logging.info("\nGenome distribution:")
logging.info(neigh_df["genome_id"].value_counts().to_string())

# --------------------------------------------------------
# SAVE INTERMEDIATE RESULTS
# --------------------------------------------------------
logging.info("\nSaving neighborhood data...")
STEP += 1

escrt_neighborhoods_file = os.path.join(
    MAIN_OUTDIR,
    f"[STEP:{STEP}]escrt_neighborhoods_{rightnow}.csv"
)
neigh_df.to_csv(escrt_neighborhoods_file, index=False)
logging.info("Saved: %s", escrt_neighborhoods_file)



# --------------------------------------------------------
# STEP 4: Merge with taxonomy
# --------------------------------------------------------
logging.info("\n" + "=" * 70)
logging.info("ADDING TAXONOMY INFORMATION")
logging.info("=" * 70)

STEP += 1
out_csv = os.path.join(
    MAIN_OUTDIR,
    f"[STEP:{STEP}]escrt_neighborhoods_with_taxonomy_{rightnow}.csv"
)

df = merge_taxonomy(
    escrt_neighborhoods_file,
    "/home/anirudh/synteny/ar53_taxonomy_r226.tsv",
    out_csv
)

logging.info("Saved taxonomy-annotated neighborhoods: %s", out_csv)

# --------------------------------------------------------
# TAXONOMIC SUMMARY
# --------------------------------------------------------
logging.info("\nTaxonomic diversity in results:")
logging.info("Unique phyla: %d", df["phylum"].nunique())
logging.info("Unique classes: %d", df["class"].nunique())
logging.info("Unique orders: %d", df["order"].nunique())

logging.info("\nSample genomes with taxonomy:")
logging.info(
    df[
        ["genome_id_base", "phylum", "class", "order"]
    ]
    .drop_duplicates()
    .head(10)
    .to_string()
)


contigs = (
    df
    .groupby(["genome_id_x", "contig"], as_index=False)
    .agg(
        neighborhood_architecture_compressed=(
            "neighborhood_architecture_compressed", "first"
        )
    )
)

logging.info("\nContig architecture summary (first 10):")
logging.info(contigs.head(10).to_string())

contigs.to_csv(os.path.join(MAIN_OUTDIR, f"[STEP:{STEP}.5]contig_architecture_summary_{rightnow}.csv"), index=False)

logging.info("\n" + "=" * 70)
logging.info("ALL PROCESSING COMPLETE")
logging.info("=" * 70)


2026-01-16 20:11:38,724 [INFO] ======================================================================
2026-01-16 20:11:38,725 [INFO] STARTING SYNTENY PIPELINE
2026-01-16 20:11:38,725 [INFO] ======================================================================
2026-01-16 20:11:38,726 [INFO] Hits DataFrame: (3325, 10)
2026-01-16 20:11:38,726 [INFO] GFF directory: /home/anirudh/genomes/selected_genomes/prokka_results
2026-01-16 20:11:38,727 [INFO] Window size: ±5 genes
2026-01-16 20:11:38,727 [INFO] 
[STEP 1] Loading GFF files...
2026-01-16 20:11:38,728 [INFO] Loading GFFs for 129 unique genomes
2026-01-16 20:11:38,728 [INFO] Parsing GFF: GCA_021498095.1_ASM2149809v1_genomic.gff
2026-01-16 20:11:38,746 [INFO]   → Parsed 3694 CDS features across 1 contigs
2026-01-16 20:11:38,749 [INFO] Parsing GFF: GCA_005191425.1_ASM519142v1_genomic.gff
2026-01-16 20:11:38,762 [INFO]   → Parsed 3164 CDS features across 182 contigs
2026-01-16 20:11:38,766 [INFO] Parsing GFF: GCA_048882285.1_ASM4888228v1_g

In [153]:
# ------------------------------------------------------------------
# DIAGNOSTIC: Check which architectures would be filtered by keywords
# ------------------------------------------------------------------

keyword_pattern = "|".join(CORE_TARGETS)

# Create a boolean mask for architectures matching keywords
matches_keywords = (
    df_with_arch["architecture"]
    .astype(str)
    .str.lower()
    .str.contains(keyword_pattern)
)

# Count what would be kept vs discarded
kept_count = matches_keywords.sum()
discarded_count = (~matches_keywords).sum()

logging.info("\n" + "=" * 70)
logging.info("ARCHITECTURE FILTERING DIAGNOSTIC")
logging.info("=" * 70)
logging.info(f"Total proteins with valid HMM hits (i_evalue ≤ 1e-5, coverage ≥ 0.65): {len(df_with_arch)}")
logging.info(f"Proteins with architectures matching keywords: {kept_count}")
logging.info(f"Proteins with architectures NOT matching keywords: {discarded_count}")

if discarded_count > 0:
    logging.warning(f"\n⚠️  WARNING: {discarded_count} proteins will be EXCLUDED from neighborhoods:")
    excluded = df_with_arch[~matches_keywords][["target", "architecture"]].drop_duplicates()
    logging.warning(excluded.head(20).to_string())
    
    # Count distribution of excluded architectures
    logging.warning("\nDistribution of excluded architectures:")
    arch_counts = df_with_arch[~matches_keywords]["architecture"].value_counts()
    logging.warning(arch_counts.head(20).to_string())
    
    # Save for review
    excluded_file = os.path.join(MAIN_OUTDIR, f"[DIAGNOSTIC]excluded_proteins_by_keyword_filter_{rightnow}.csv")
    df_with_arch[~matches_keywords][["target", "architecture", "architecture_method"]].drop_duplicates().sort_values(by = ['architecture'], ascending=False).to_csv(excluded_file, index=False)
    logging.warning(f"\nSaved excluded proteins to: {excluded_file}")
else:
    logging.info("✓ All proteins with valid hits match keyword filter - no data loss")

logging.info("=" * 70)

2026-01-16 20:12:04,913 [INFO] 
2026-01-16 20:12:04,914 [INFO] ARCHITECTURE FILTERING DIAGNOSTIC
2026-01-16 20:12:04,914 [INFO] ======================================================================
2026-01-16 20:12:04,915 [INFO] Total proteins with valid HMM hits (i_evalue ≤ 1e-5, coverage ≥ 0.65): 3325
2026-01-16 20:12:04,915 [INFO] Proteins with architectures matching keywords: 1470
2026-01-16 20:12:04,915 [INFO] Proteins with architectures NOT matching keywords: 1855
2026-01-16 20:12:04,916 [WARNING] 
⚠️  WARNING: 1855 proteins will be EXCLUDED from neighborhoods:
2026-01-16 20:12:04,919 [WARNING]             target architecture
0   AAEOKEPF_00166           E2
3   AAEOKEPF_00273       E3-UFM
6   AAEOKEPF_00322           E1
8   AAEOKEPF_00337           E1
10  AAEOKEPF_00338          MPN
12  AAEOKEPF_00690          MPN
13  AAEOKEPF_00878          MPN
15  AAEOKEPF_00882      Ub-like
19  AAEOKEPF_01109          MPN
20  AAEOKEPF_01385          MPN
29  AAEOKEPF_01448          MPN
38  AAE